Do one of the following exercises using requests and BeautifulSoup libraries


1) Web scrape flexcar.gr
Get the features (brand, model, price, hp, gearbox, extras..) for all leasing car offers.  

2) Web scrape ATPWorldTour website (singles)
Get all weeks (from 1973 to today) of the top100 rankings, scrape date, player name, ranking, country and points and store the data to a dataframe and save it at the end to a csv file. (Hint: use an empty string “” in the headers)  

headers={'User-Agent': ''}  
page = requests.get(url,timeout=15, headers= headers)  

Bonus:  
3) Web scrape kariera.gr using Selenium
Retrieve all job ads for Data Analyst, Data Scientist and Data Engineer and store to a dataframe features like: Company, Job title, Content, Location and job occupation.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "https://www.coffeereview.com/review/"
response = requests.get(url)
response.status_code
# Check status code, we want 200
if response.status_code != 200:
    print("Error")

In [3]:
soup = BeautifulSoup(response.content, 'html.parser')
type(soup)

bs4.BeautifulSoup